# Entendendo o calculo por trás do Prophet

O Prophet é uma ferramenta de código aberto do Facebook usada para prever dados de séries temporais que ajuda as empresas a entender e possivelmente prever o mercado. É baseado em um modelo aditivo decomposto onde as tendências não lineares se ajustam à sazonalidade, e também leva em consideração os efeitos dos feriados. Antes de seguirmos diretamente para a codificação, vamos aprender alguns termos necessários para entender isso.

Tendência:
a tendência mostra a tendência dos dados de aumentar ou diminuir ao longo de um longo período de tempo e filtra as variações sazonais.

Sazonalidade:
Sazonalidade são as variações que ocorrem em um curto período de tempo e não são proeminentes o suficiente para serem chamadas de “tendência”.

Compreendendo o modelo do profeta
A ideia geral do modelo é semelhante a um modelo aditivo generalizado . A “Equação do Profeta” se encaixa, conforme mencionado acima, tendência, sazonalidade e feriados. Isso é dado por,

y (t) = g (t) + s (t) + h (t) + e (t)


Onde,

g (t) refere-se à tendência (mudanças ao longo de um longo período de tempo)

s (t) refere-se à sazonalidade (mudanças periódicas ou de curto prazo)

h (t) refere-se a efeitos de feriados para a previsão

e (t) refere-se às mudanças incondicionais que são específicas de um negócio, pessoa ou circunstância. Também é chamado de termo de erro.

y (t) é a previsão.

## Imports

In [0]:
pip install prophet

Python interpreter will be restarted.
  Using cached holidays-0.39-py3-none-any.whl (869 kB)
  Attempting uninstall: holidays
    Found existing installation: holidays 0.11.3.1
    Not uninstalling holidays at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-92448fc8-1b8f-4ab0-b6de-7e57f8325cd7
    Can't uninstall 'holidays'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
verstack 3.9.0 requires holidays==0.11.3.1, but you have holidays 0.39 which is incompatible.
Python interpreter will be restarted.


In [0]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import scipy as scipy
import numpy as np

In [0]:
from pyspark.sql.functions import *

## Base

In [0]:
### Dataframe utilizado

In [0]:
###O Prophet prevê dados apenas quando eles estão em um determinado formato. O dataframe com os dados deve ter a coluna salva como ds para os dados da série temporal ey para os dados a serem previstos.

df = spark.sql('''
SELECT
''')
df.display()

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-4117247203547461>:3
      1 ###O Prophet prevê dados apenas quando eles estão em um determinado formato. O dataframe com os dados deve ter a coluna salva como ds para os dados da série temporal ey para os dados a serem previstos.
----> 3 df = spark.sql('''
      4 SELECT
      5 ''')
      6 df.display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_

In [0]:
df.describe().show()

## Tratamento da base com prophet

In [0]:
### Conversão o dataframe de pySpark para Pandas

In [0]:
pandasDF = df.toPandas()
pandasDF.display()

## Tratamento da base com prophet

In [0]:
fig = plt.figure(figsize=(15,5))
ax = fig.gca()
pandasDF.plot.hist(bins=40, ax=ax)


In [0]:
pandasDF1 = pandasDF["y"].dropna()
scipy.stats.shapiro(pandasDF1)

In [0]:
### Ajustamos o modelo instanciando um novo Prophetobjeto. Quaisquer configurações para o procedimento de previsão são passadas para o construtor. Então você chama seu fitmétodo e passa no dataframe histórico. A montagem deve levar de 1 a 5 segundos.

In [0]:
m = Prophet()
m.fit(pandasDF)

In [0]:
### As previsões são feitas em um dataframe com uma coluna dscontendo as datas para as quais uma previsão deve ser feita. Você pode obter um dataframe adequado que se estende no futuro por um número especificado de dias usando o método auxiliar Prophet.make_future_dataframe. Por padrão, ele também incluirá as datas do histórico, para que possamos ver o ajuste do modelo também.

In [0]:
### Queremos uma previsão para um ano. A frequência de nossos dados é de 1 mês e, portanto, para 2 anos, é 12 * 2 = 24 meses. Portanto, precisamos adicionar 12 a mais linhas de dados mensais a um dataframe.

In [0]:
future = m.make_future_dataframe(periods= 90, freq='D')
future.tail()

In [0]:
# Na tabela ds , como sabemos, estão os dados da série temporal. yhat é a previsão, yhat_lower e yhat_upper são os níveis de incerteza (basicamente significa que a previsão e os valores reais podem variar dentro dos limites dos níveis de incerteza). Em seguida, temos a tendência que mostra o crescimento de longo prazo, redução ou estagnação dos dados, trend_lower e trend_upper são os níveis de incerteza 

In [0]:
coisa = m.predict(future)
coisa[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail(90)

In [0]:
coisa["mes"] = pd.DatetimeIndex(coisa["ds"]).month
coisa.reset_index(inplace = True)

In [0]:
coisa.groupby("mes").sum()


In [0]:
### Você pode plotar a previsão chamando o Prophet.plotmétodo e passando seu dataframe de previsão.
# A imagem abaixo mostra a previsão básica. O azul claro é o nível de incerteza ( yhat_upper e yhat_lower ), o azul escuro é a previsão ( yhat ) e os pontos pretos são os dados originais. 

In [0]:
fig1 = m.plot(forecast)

In [0]:
### Se você quiser ver os componentes da previsão, pode usar o Prophet.plot_componentsmétodo. As imagens abaixo mostram as tendências e sazonalidade (em um ano) dos dados da série temporal.

In [0]:
fig2 = m.plot_components(forecast)

In [0]:
### Uma figura interativa da previsão e componentes pode ser criada com plotly. 

In [0]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [0]:
plot_components_plotly(m, forecast)

## Testes

### Teste 1 : filtrar os últimos 12 meses de 2022-07-01 a 2023-07-31

#### prever 3 meses

In [0]:
future3 = m.make_future_dataframe(periods= 90, freq='D')
future3.tail()

In [0]:
forecast = m.predict(future3)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()

In [0]:
fig1 = m.plot(forecast)

#### prever 6 meses

In [0]:
future6 = m.make_future_dataframe(periods= 180, freq='D')
future6.tail()

In [0]:
forecast = m.predict(future6)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()

#### prever só 9 meses

In [0]:
future9 = m.make_future_dataframe(periods= 270, freq='D')
future9.tail()

In [0]:
forecast = m.predict(future9)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()

#### prever só 12 meses

In [0]:
future12 = m.make_future_dataframe(periods= 365, freq='D')
future12.tail()

In [0]:
forecast = m.predict(future12)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()

In [0]:
###O Prophet prevê dados apenas quando eles estão em um determinado formato. O dataframe com os dados deve ter a coluna salva como ds para os dados da série temporal ey para os dados a serem previstos.

df_abril = spark.sql('''
SELECT
''')
df_abril.display()

In [0]:
df_abril.groupBy(trunc('ds', 'month')).agg(sum('y')).display()

In [0]:
future = m.make_future_dataframe(periods= 90, freq='D')
future.tail()

In [0]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']].tail()